In [1]:
from utils import utils
import pandas as pd
import numpy as np
from tensorflow import keras

In [2]:
ufc = pd.read_csv('../data/external/8-21-2021/upcoming-event (1).csv')

In [3]:
ufc.head()

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
0,Jared Cannonier,Kelvin Gastelum,-150.0,130.0,66.666667,130.000000,2021-08-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,275.0,250.0,1600.0,1100.0,165.0,600.0
1,Clay Guida,Mark Madsen,140.0,-160.0,140.000000,62.500000,2021-08-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,275.0,120.0,1000.0,550.0,700.0,650.0
2,Parker Porter,Chase Sherman,160.0,-190.0,160.000000,52.631579,2021-08-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,450.0,300.0,800.0,1600.0,450.0,120.0
3,Trevin Jones,Saidyokub Kakhramonov,NaN,NaN,NaN,NaN,2021-08-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Vinc Pichel,Austin Hubbard,-105.0,-115.0,95.238095,86.956522,2021-08-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,140.0,180.0,2000.0,1200.0,600.0,450.0


In [4]:
X = utils.transform(ufc)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 11
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   weight_class            12 non-null     object  
 1   gender                  12 non-null     category
 2   better_rank             12 non-null     category
 3   lose_streak_dif         12 non-null     category
 4   win_streak_dif          12 non-null     category
 5   longest_win_streak_dif  12 non-null     category
 6   win_dif                 12 non-null     int64   
 7   loss_dif                12 non-null     int64   
 8   total_round_dif         12 non-null     int64   
 9   total_title_bout_dif    12 non-null     category
 10  ko_dif                  12 non-null     category
 11  sub_dif                 12 non-null     int64   
 12  height_dif              12 non-null     float64 
 13  reach_dif               12 non-null     float64 
 14  age_dif                 12 n

In [5]:
model = keras.models.load_model('../models/tf_models/model_1/')

In [12]:
winners = (model.predict(X) > 0.5).astype("int32")

In [29]:
preds = ufc[['R_fighter', 'B_fighter']].copy()
preds['winner_pred'] = winners

for i, r in preds.iterrows():
    if r[2] == 1:
        preds['winner_pred'].iloc[i] = r[0]
    else:
        preds['winner_pred'].iloc[i] = r[1]
        
preds.columns = ['Red Corner', 'Blue Corner', 'Predicted Winner']

C:\Users\jpzam\miniconda3\envs\tf_gpu\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [31]:
preds.to_csv('../data/interim/8-21-2021/predictions.csv', index=False)